In [ ]:
class VGG16Transfer(NNClassifier):

    def __init__(self, num_classes, fine_tuning=False):
        super(VGG16Transfer, self).__init__()
        vgg = tv.models.vgg16_bn(pretrained=True)
        for param in vgg.parameters():
            param.requires_grad = fine_tuning
        self.features = vgg.features
        self.classifier = vgg.classifier
        num_ftrs = vgg.classifier[6].in_features
        self.classifier[6] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        f = self.features(x).view(x.shape[0], -1)       
        y = self.classifier(f)
        return y

In [ ]:
class VGG16(Classifier):

    def __init__(self, num_classes, fine_tuning=False):
        super(VGG16, self).__init__()
        
        # Use the new weights argument instead of deprecated pretrained=True
        vgg = tv.models.vgg16_bn(weights=VGG16_BN_Weights.DEFAULT)

        # Freeze or unfreeze all parameters depending on fine_tuning flag
        for param in vgg.parameters():
            param.requires_grad = fine_tuning

        # Keep the convolutional feature extractor
        self.features = vgg.features
        
        # Copy the classifier
        self.classifier = vgg.classifier
        
        # Replace the final fully-connected layer to match number of classes
        num_ftrs = vgg.classifier[6].in_features
        self.classifier[6] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # Extract features
        f = self.features(x).view(x.shape[0], -1)
        # Classify
        y = self.classifier(f)
        return y

# Spare Function 2

In [ ]:
class ClassificationStatsManager:
    def __init__(self):
        self.init()

    def init(self):
        # Reset tracking variables
        self.running_loss = 0.0
        self.running_accuracy = 0.0
        self.number_update = 0

    def accumulate(self, loss, x, y, d):
        # Accumulate total loss
        self.running_loss += loss.item()

        # Get predictions (highest logit per sample)
        _, predicted = torch.max(y, dim=1)

        # Compute accuracy for this batch
        batch_accuracy = (predicted == d).float().mean().item()

        # Accumulate totals
        self.running_accuracy += batch_accuracy
        self.number_update += 1

    def summarize(self):
        # Compute average loss and accuracy
        avg_loss = self.running_loss / self.number_update
        avg_acc = 100.0 * self.running_accuracy / self.number_update
        return {'loss': avg_loss, 'accuracy': avg_acc}


In [ ]:
class ClassificationStatsManager(nt.StatsManager):

    def __init__(self):
        super(ClassificationStatsManager, self).__init__()

    def init(self):
        super(ClassificationStatsManager, self).init()
        self.running_accuracy = 0

    def accumulate(self, loss, x, y, d):
        super(ClassificationStatsManager, self).accumulate(loss, x, y, d)
        _, l = torch.max(y, 1)
        self.running_accuracy += torch.mean((l == d).float())

    def summarize(self):
        loss = super(ClassificationStatsManager, self).summarize()
        accuracy = 100 * self.running_accuracy / self.number_update
        return {'loss': loss, 'accuracy': accuracy}

# Spare Function 3

In [ ]:
from torch.utils.data import DataLoader

# --- Setup ---
lr = 1e-3
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create model
net = VGG16(num_classes)
net = net.to(device)

# Optimizer and loss function
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

# Data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Stats manager
stats_manager = ClassificationStatsManager()

# --- Training Loop (replaces nt.Experiment) ---
epochs = 10
for epoch in range(epochs):
    net.train()
    stats_manager.init()

    for x, d in train_loader:
        x, d = x.to(device), d.to(device)
        
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, d)
        loss.backward()
        optimizer.step()

        stats_manager.accumulate(loss, x, y, d)

    train_stats = stats_manager.summarize()
    print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_stats['loss']:.4f}, Train Acc: {train_stats['accuracy']:.2f}%")

    # --- Validation ---
    net.eval()
    stats_manager.init()
    with torch.no_grad():
        for x, d in val_loader:
            x, d = x.to(device), d.to(device)
            y = net(x)
            loss = criterion(y, d)
            stats_manager.accumulate(loss, x, y, d)

    val_stats = stats_manager.summarize()
    print(f"           Val Loss: {val_stats['loss']:.4f}, Val Acc: {val_stats['accuracy']:.2f}%")


In [ ]:
lr = 1e-3
net = VGG16Transfer(num_classes)
net = net.to(device)
adam = torch.optim.Adam(net.parameters(), lr=lr)
stats_manager = ClassificationStatsManager()
exp1 = nt.Experiment(net, train_set, val_set, adam, stats_manager,
               output_dir="birdclass1", perform_validation_during_training=True)

# Spare Function 4

In [ ]:
# Split the Training Dataset into Training and Val Datasets
def split_train_val(df, target_column="class", val_ratio=0.2, random_state=20, stratify=True):
# random_state is the random seed for the splitting of training and val images
# stratify preserve class distribution
    
    stratify_col = df[target_column] if stratify else None

    X_train, X_val, y_train, y_val = train_test_split(
        df['file_path'],
        df[target_column],
        test_size=val_ratio,
        random_state=random_state,
        stratify=stratify_col
    )

    return X_train, X_val, y_train, y_val


def make_dataset_df(root_dir):
data = []
for cls in os.listdir(root_dir):
    cls_path = os.path.join(root_dir, cls)
    if os.path.isdir(cls_path):
        for img_name in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img_name)
            data.append({'file_path': img_path, 'class': cls})
return pd.DataFrame(data)

# Spare Function 5

In [1]:
class BirdDataset(td.Dataset): #td.Dataset is a class from td (torch.utils.data)

    # Constructor [Param: root file, train (training or testing dataset), download (online? if local x exist), target & label transform]

    # Custom Dataset uses different Constructor [Param up to dev]
    def __init__(self, root_dir, mode="train", image_size=(224,224) ):
        super().__init__() # Call Parent __init__ function, ensure proper inheritance. (Newed Python)
        self.image_size = image_size # Create new variable to store current self image_size 
        self.mode = mode
        # Loading Dataset from Train Folder     
        self.images_dir = os.path.join(dataset_root_dir, "Train")
        ## Temp: Should Return "E:\Year 3 Sem 1\COS30082 Applied Machine Learning\Assignment_1\Dataset\Train"
        
        # Loading .txt file (train.txt)
        # Change to Local File Path
        txt_path = os.path.join (root_dir, f'{mode}.txt')
        print("Text File:", txt_path)

        
        self.data = self.data = pd.read_csv(txt_path, sep=" ", header=None, names=["file_path", "class"])

    
    # Return No. of Data Images
    def __len__(self): # __len__ is a method with built-in len() function
        # print("Dataset Length:", len(self.data))
        return len(self.data)
    
    
    # Return Init Configuration
    def __repr__(self): 
        # __repr__ is a method to represent an object as a string. Images have x and y. Not using repr would result in the return of the memory location instead ( 0x10e104570)
        # Can represent other things too from a memory location
        
        return "BirdDataset: (mode='{}', image_size={})".format(self.mode, self.image_size)

    # Preparing the Images
    def __getitem__(self,idx):
        # Method automatically called, Allows class to behave like list or arrays
        # Used to call and "get" datasets in the form of image and label
        # Index 'idx'

        # Image Path
        img_path = os.path.join(self.images_dir, self.data.iloc[idx]['file_path'])
        ## Temp: Should Return "E:\Year 3 Sem 1\COS30082 Applied Machine Learning\Assignment 1\Dataset\Train" + "Specific Image Location"

        # Load Image
        img = Image.open(img_path)

        # Transformation
        transform = tv.transforms.Compose([
            # Resize the Image
            tv.transforms.Resize(self.image_size),

            # Convert it to Tensor
            tv.transforms.ToTensor(),

            # Normalize it to the standard range (1, -1)
            tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

            # C, H and W [Channel(Most common color channels), Height, Width]
            # 1st tuple -> Mean for (R,G,B), 2nd tuple -> S.T.D for (R,G,B)
        ])

        # Processed Image    
        x = transform(img)

        # Get Class Label from Train.txt file
        d = self.data.iloc[idx]['class']
        
        return x, d 
        
    def number_of_classes(self):
        return self.data['class'].max() + 1
    # Max is used to find the largest class number (Classes are represented by numbers)
    # Max + 1 because the index starts from 0 (To Compensate).
        

NameError: name 'td' is not defined

# Classifier (Original)

In [ ]:
class VGG16(Classifier):

    def __init__(self, num_classes, fine_tuning=False):
        super(VGG16, self).__init__()
        vgg = tv.models.vgg16_bn(pretrained=True)
        for param in vgg.parameters():
            param.requires_grad = fine_tuning
        self.features = vgg.features
        self.classifier = vgg.classifier
        num_ftrs = vgg.classifier[6].in_features
        self.classifier[6] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        f = self.features(x).view(x.shape[0], -1)       
        y = self.classifier(f)
        return y

# Freezing (Mod)

In [ ]:
# Freeze first N layers
for i, param in enumerate(self.features.parameters()):
    if i < 20:  # freeze first 20 parameters (adjustable)
        param.requires_grad = False


# Weight Decay in Optimizer (Mod)

In [ ]:
import torch.optim as optim

lr = 1e-4
weight_decay = 1e-4  # L2 regularization
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)


# Early Stopping & LR Scheduling (Mod)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
early_stopper = EarlyStopping(patience=5)

for epoch in range(start_epoch, num_epochs):
    # training step ...
    
    val_loss = self.evaluate()[0]['loss']
    
    # Step the scheduler
    scheduler.step(val_loss)
    
    # Early stopping
    early_stopper.step(val_loss)
    if early_stopper.stop:
        print(f"Early stopping at epoch {epoch+1}")
        break


In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(7, 3))
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
num_epochs = 20

for epoch in range(num_epochs):
    exp1.run(num_epochs=1, plot=lambda exp: plot(exp, fig=fig, axes=axes))
    
    val_loss = exp1.history[-1][1]['loss']  # validation loss
    early_stopping.step(val_loss)
    
    if early_stopping.should_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

In [ ]:
class BirdDataset(td.Dataset):  # td.Dataset is torch.utils.data.Dataset
    def __init__(self, root_dir, mode="train", image_size=(224,224), transform=None):
        super().__init__()
        self.image_size = image_size
        self.mode = mode

        # ✅ ADD THIS LINE (Fixes your AttributeError)
        self.transform = transform   

        # Loading Dataset from Train Folder
        self.images_dir = os.path.join(root_dir, "Train")

        # Load .txt file (train.txt / val.txt)
        txt_path = os.path.join(root_dir, f'{mode}.txt')
        print("Text File:", txt_path)
        self.data = pd.read_csv(txt_path, sep=" ", header=None, names=["file_path", "class"])

        # ✅ REMOVE THESE if you plan to pass transforms externally
        # self.train_transform = ...
        # self.val_transform = ...

        # ✅ OPTIONAL: add default transform (only used if none provided)
        self.default_transform = tv.transforms.Compose([
            tv.transforms.Resize(self.image_size),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        return f"BirdDataset(mode='{self.mode}', image_size={self.image_size})"

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.data.iloc[idx]['file_path'])
        img = Image.open(img_path).convert('RGB')
        label = self.data.iloc[idx]['class']

        # ✅ CHANGE THIS SECTION
        # Use provided transform if available, else fallback to default
        if self.transform:
            img = self.transform(img)
        else:
            img = self.default_transform(img)

        return img, label

    def number_of_classes(self):
        return self.data['class'].max() + 1


In [ ]:

class Resnet18Transfer(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        for param in self.model.parameters():
            param.requires_grad = True  # fine-tune all layers

        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),  # reduce overfitting
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
